In [ ]:
!pip install qiskit
!pip install qiskit-ibm-runtime
!pip install qiskit-aer

In [ ]:

# 최초 1회만 실행하면 됩니다.
from qiskit_ibm_runtime import QiskitRuntimeService

token="Your IBM Quantum API token here"

service = QiskitRuntimeService.save_account(
  token=token,
  channel="ibm_quantum_platform",
  overwrite=True
  )

In [1]:
# 위의 작업을 완료한 후에는 아래의 코드를 사용하여 IBM Quantum 서비스에 연결할 수 있습니다.
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-06-24 08:55:04,011: Default instance not set. Searching all available instances.


In [6]:
from qiskit import QuantumCircuit
from qiskit_aer.primitives import SamplerV2

# 1개의 큐비트와 1개의 고전 비트 레지스터를 생성합니다.
circuit = QuantumCircuit(1, 1) # QuantumCircuit(큐비트 수, 고전 비트 수)

# 큐비트에 Hadamard 게이트를 적용하여 중첩 상태로 만듭니다.
circuit.h(0)

# 큐비트를 측정하고 결과를 고전 비트에 저장합니다.
circuit.measure(0, 0)

# Sampler 프리미티브를 초기화합니다. (기본적으로 Aer 시뮬레이터를 사용합니다.)
sampler = SamplerV2()

# 회로를 Sampler에 제출하고 실행합니다.
# shots=1은 1번만 측정하여 하나의 난수 비트를 얻겠다는 의미입니다.
job = sampler.run([circuit], shots=1) # SamplerV2 run method expects a list of circuits
result = job.result()
pub_result = result[0]

counts = pub_result.data.c.get_counts()
print(counts)

# 결과 출력
random_bits_str = list(counts.keys())[0]
print(random_bits_str)


{'0': 1}
0


In [8]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler import generate_preset_pass_manager

# 1개의 큐비트와 1개의 고전 비트 레지스터를 생성합니다.
circuit = QuantumCircuit(1, 1) # QuantumCircuit(큐비트 수, 고전 비트 수)

# 큐비트에 Hadamard 게이트를 적용하여 중첩 상태로 만듭니다.
circuit.h(0)

# 큐비트를 측정하고 결과를 고전 비트에 저장합니다.
circuit.measure(0, 0)

backend = service.least_busy(simulator=False, operational=True)

sampler = Sampler(mode=backend)
# Get the target from the backend
target = backend.target

# Generate a pass manager for transpilation
pass_manager = generate_preset_pass_manager(target=target, optimization_level=1)

# Transpile the circuit
transpiled_circuit = pass_manager.run(circuit)

# Now run the transpiled circuit on the Sampler
# job = sampler.run([circuit], shots=1)
job = sampler.run([transpiled_circuit], shots=1)
print(f">>> Job ID: {job.job_id()}")

>>> Job ID: d19veumpmg9c73956qvg


In [ ]:
# Retrieve the job result
result = job.result()

pub_result = result[0]

# Get the counts from the result
counts = pub_result.data.c.get_counts()

# Print the results (should be either 0 or 1)
print("측정 결과:", counts)

# 결과 출력b
random_bits_str = list(counts.keys())[0]
print(random_bits_str)

측정 결과: {'1': 1}
1


In [ ]:
import random

def get_permutation_from_single_random_number(random_rank):
    """
    하나의 난수(rank)를 사용하여 1부터 7까지의 순열을 생성합니다.
    random_rank는 0부터 5039까지의 정수여야 합니다.
    7! 가지 = 7P7 = 5040 가지의 순열이 존재합니다.
    0 -> (1, 2, 3, 4, 5, 6, 7) 
    1 -> (1, 2, 3, 4, 5, 7, 6)
    f(n) = (a1, a2, a3, a4, a5, a6, a7)

    a7 = n // 6!
    b7 = n % 6!
    a6 = b7 // 5!
    b6 = b7 % 5!
    a5 = b6 // 5!
    b5 = b6 % 5!
    a4 = b5 // 5!
    b4 = b5 % 5!
    a3 = b4 // 5!
    b3 = b4 % 5!

    """

    n_elements = 7
    # 1부터 n_elements까지의 숫자를 리스트로 만듭니다.
    elements = list(range(1, n_elements + 1)) # [1, 2, 3, 4, 5, 6, 7]
    permutation = []

    # 팩토리얼 값 미리 계산 (디랭크 알고리즘에 필요)
    # factorial[0] = 1, factorial[1] = 1, factorial[2] = 2, ..., factorial[6] = 720
    factorials = [1] * (n_elements + 1) # [1, 1, 1, 1, 1, 1, 1, 1]
    for i in range(2, n_elements + 1):
        factorials[i] = factorials[i-1] * i

    current_rank = random_rank

    for i in range(n_elements - 1, -1, -1): # 6부터 0까지 반복
        # 현재 팩토리얼 자리수 (i!)로 나눈 몫이 인덱스가 됩니다.
        index = current_rank // factorials[i]

        # 해당 인덱스의 숫자를 결과 순열에 추가하고 원본 리스트에서 제거
        permutation.append(elements.pop(index))

        # 나머지 랭크를 다음 반복을 위해 갱신합니다.
        current_rank %= factorials[i]

    return permutation

# ----------------------------------------------------------------------
# 사용 예시:
# 1. 0부터 5039 사이의 난수 한 개를 생성합니다.
#    (이 예시에서는 편의상 random.randint를 사용하지만,
#     실제 시스템의 0 ~ 2^n-1 난수를 0 ~ 5039로 변환하는 과정이 중요합니다.)

# 예시 1: 특정 난수 값 (0은 오름차순 정렬된 순열을 의미)
single_random_number = 0
shuffled_result = get_permutation_from_single_random_number(single_random_number)
print(f"난수 {single_random_number}로 생성된 순열: {shuffled_result}")
# 예상 출력: [1, 2, 3, 4, 5, 6, 7]

# 예시 2: 다른 특정 난수 값 (5039는 내림차순 정렬된 순열을 의미)
single_random_number = 5039
shuffled_result = get_permutation_from_single_random_number(single_random_number)
print(f"난수 {single_random_number}로 생성된 순열: {shuffled_result}")
# 예상 출력: [7, 6, 5, 4, 3, 2, 1]

# 예시 3: 무작위 난수 값
# (실제 환경에서는 여기서 0 ~ 2^n-1 난수를 받아서 편향 없이 0 ~ 5039로 변환해야 합니다.)
rand_val_for_example = random.randint(0, 5039)
shuffled_result = get_permutation_from_single_random_number(rand_val_for_example)
print(f"난수 {rand_val_for_example}로 생성된 순열: {shuffled_result}")

난수 0로 생성된 순열: [1, 2, 3, 4, 5, 6, 7]
난수 5039로 생성된 순열: [7, 6, 5, 4, 3, 2, 1]
난수 4486로 생성된 순열: [7, 2, 3, 6, 5, 1, 4]


In [4]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2 as Sampler

# 3개의 큐비트와 3개의 고전 비트 레지스터를 생성합니다.
num_bits = 13
circuit = QuantumCircuit(num_bits, num_bits) # QuantumCircuit(큐비트 수, 고전 비트 수)

# 각 큐비트에 Hadamard 게이트를 적용하여 중첩 상태로 만듭니다.
for i in range(num_bits):
    circuit.h(i)

# 모든 큐비트를 측정하고 결과를 해당 고전 비트에 저장합니다.
# measure(source_qubits, target_classical_bits)
circuit.measure(range(num_bits), range(num_bits))

backend = service.least_busy(simulator=False, operational=True)
sampler = Sampler(mode=backend)

from qiskit.transpiler import generate_preset_pass_manager

# Get the target from the backend
target = backend.target

# Generate a pass manager for transpilation
pass_manager = generate_preset_pass_manager(target=target, optimization_level=1)

# Transpile the circuit
transpiled_circuit = pass_manager.run(circuit)

job = sampler.run([transpiled_circuit], shots=1)
print(f">>> Job ID: {job.job_id()}")

>>> Job ID: d1cukck7tq0c73ddu060


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

instance = 'URN' # 예시: 'urn:ibm/quantum/instance/1234567890abcdef'
job_id = 'ID'

service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    instance=instance
)
job = service.job(job_id)

counts = job.result()[0].data.c.get_counts()

# 결과 출력
random_bits_str = list(counts.keys())[0]

# 만약 int형으로 변환하고 싶다면:
random_integer = int(random_bits_str, 2)
print(f"정수 형태: {random_integer}")
if random_integer < 5040:
  shuffled_result = get_permutation_from_single_random_number(random_integer)
  team_list = list(map(lambda x: x + 8, shuffled_result))
  print(team_list)
else:
  print("난수 값이 0 ~ 5039 범위를 벗어났습니다. 재시도하세요.")

In [1]:
!pip install pennylane

  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.4 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.8 MB/s eta 0:00:00
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.3 MB/s eta 0:00:00a 0:00:01
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached gast-0.6.0-py3-none-any.w

In [3]:
import pennylane as qml
from pennylane import numpy as np

# 1. PennyLane 장치 설정 (시뮬레이터)
# 'lightning.qubit'은 PennyLane의 고성능 C++ 기반 시뮬레이터입니다.
# wires=3은 3개의 큐비트를 사용하겠다는 의미입니다.
dev = qml.device("lightning.qubit", wires=3, shots=1) # shots=1은 한 번의 실행으로 3비트 난수열을 얻겠다는 의미입니다.

# 2. 양자 회로 정의
# @qml.qnode 데코레이터는 이 함수가 양자 회로임을 나타냅니다.
@qml.qnode(dev, interface="autograd") # 'interface="autograd"'는 자동 미분 엔진을 지정합니다.
def quantum_random_number_generator():
    # 각 큐비트에 Hadamard 게이트를 적용하여 중첩 상태로 만듭니다.
    for i in range(3):
        qml.Hadamard(wires=i)

    # 모든 큐비트를 측정하고 결과를 반환합니다.
    # qml.sample()은 장치에서 회로를 실행하고 측정 샘플을 반환합니다.
    # shots=1이므로 한 번의 실행으로 3비트 난수열을 얻습니다.
    return qml.sample(wires=[0, 1, 2])

# 3. 회로 실행 및 결과 가져오기
# quantum_random_number_generator()를 호출하여 회로를 실행합니다.
# shots=1이므로 결과는 [0, 1]과 같은 배열 형태가 아닌 단일 샘플 배열로 반환됩니다.
random_bits_array = quantum_random_number_generator()

# PennyLane의 qml.sample()은 NumPy 배열로 결과를 반환합니다.
# 결과를 이진 문자열로 변환합니다.
random_bits_str = "".join(str(bit) for bit in random_bits_array)

print(f"생성된 3비트 난수열 (PennyLane): {random_bits_str}")

# 만약 int형으로 변환하고 싶다면:
# random_integer = int(random_bits_str, 2)
# print(f"정수 형태: {random_integer}")

생성된 3비트 난수열 (PennyLane): 101


Test
$x$
Test
$$
x = y + z
$$